In [ ]:
# This is for making changes on the fly

%load_ext autoreload
%autoreload 2

In [ ]:
%pylab inline

In [ ]:
from os.path import join
import numpy as np
from SWOTRiver import SWOTL2
from RiverObs import ReachExtractor
from RiverObs import WidthDataBase

In [ ]:
import os
from os.path import exists
def find_riverobs_test_data_dir():
    """Fin the location of the test data root directory"""
    
    if 'RIVEROBS_TESTDATA_DIR' in os.environ:
        test_data_dir = os.environ('RIVEROBS_TESTDATA_DIR')
    else: # try the default location
        test_data_dir = '../../../RiverObsTestData'
        
    if not exists(test_data_dir):
        print('You must either set the environment variable RIVEROBS_TESTDATA_DIR')
        print('or locate the test data directory at ../../../RiverObsTestData')
        raise Exception('Test data directory not found.')
        
    return test_data_dir

data_dir = find_riverobs_test_data_dir()
data_dir

In [ ]:
l2_file = join(data_dir,'L2','L2v1','swot_heights_ohio_example_v1.Multilook_L2PIXC.nc')
assert exists(l2_file)

db_dir = join(data_dir,'GRWDL')
shape_file_root = join(db_dir,'nAmerica_GRWDL_river_topo','nAmerica_GRWDL_river_topo')
db_file = join(db_dir,'nAmerica_GRWDL.h5')
assert exists(db_file)
db_file

In [ ]:
lonmin =  -83 
latmin =  38
lonmax =  -82
latmax =  39
bounding_box = lonmin,latmin,lonmax,latmax

# The list of classes to consider for potential inundation.
# The truth classes are [1], if no_layover_classification' is used.
# If estimated classification is used, the choice depends on whether
# use_fractional_inundation is set.
# If it is not set, either [3,4] or [4] should be used.
# If it is set, [2,3,4] or [3,4] should be used.
class_list = [2,3,4,5]

lat_kwd = 'latitude_medium'
lon_kwd = 'longitude_medium'
class_kwd = 'classification'
height_kwd = 'height_medium'

l2 = SWOTL2(l2_file,bounding_box=bounding_box,
            class_list=class_list,
            lat_kwd=lat_kwd,lon_kwd=lon_kwd,class_kwd=class_kwd)

In [ ]:
clip_buffer = 0.02

clip = False
reaches_no_clip = ReachExtractor(shape_file_root, l2,clip=clip,
                             clip_buffer=clip_buffer)

clip = True
reaches_clip = ReachExtractor(shape_file_root, l2,clip=clip,
                             clip_buffer=clip_buffer)

In [ ]:
print(reaches_no_clip.reach_idx, reaches_clip.reach_idx)
print(reaches_no_clip[1].lon.shape, reaches_clip[1].lon.shape)

In [ ]:
db = WidthDataBase(db_file)

In [ ]:
reach_index = reaches_no_clip.reach_idx[1]

lon_nc,lat_nc = db.get_lon_lat(reach_index)
print('no clip length: %d'%len(lon_nc))

lon_c,lat_c,inbbox = db.get_lon_lat(reach_index,
                             bounding_box=l2.bounding_box,
                             clip_buffer=clip_buffer)

print('clip length: %d'%len(lon_c))
                               
figsize(10,5)
subplot(1,2,1)
plot(lon_nc,lat_nc,'.',alpha=0.1)

subplot(1,2,2)
plot(lon_c,lat_c,'.',alpha=0.1)


In [ ]:
reach_index = reaches_no_clip.reach_idx[1]

x_nc,y_nc = db.get_xy(reach_index,l2.proj)
print('no clip length: %d'%len(x_nc))

x_c,y_c = db.get_xy(reach_index,l2.proj,
                             bounding_box=l2.bounding_box,
                             clip_buffer=clip_buffer)

print('clip length: %d'%len(x_c))
                               
figsize(10,5)
subplot(1,2,1)
plot(x_nc,y_nc,'.',alpha=0.1)

subplot(1,2,2)
plot(x_c,y_c,'.',alpha=0.1)


In [ ]:
reach_index = reaches_no_clip.reach_idx[1]

lon_nc,lat_nc,width_nc = db.get_river(reach_index,
                                      columns=['long','lat','width'],
                             asarray=True,transpose=True)
print('no clip length: %d'%len(lon_nc))

lon_c,lat_c,width_c = db.get_river(reach_index,
                                      columns=['long','lat','width'],
                             asarray=True,transpose=True,
                             bounding_box=l2.bounding_box,
                             clip_buffer=clip_buffer
                             )
print('clip length: %d'%len(lon_c))

In [ ]:
figsize(10,5)

subplot(1,2,1)
scatter(lon_nc,lat_nc,c=width_nc,edgecolor='none',alpha=0.1)

subplot(1,2,2)
scatter(lon_c,lat_c,c=width_c,edgecolor='none',alpha=0.1)